### Import libaries and dataset

In [1]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=735828 sha256=ebcab2bcf25a258acf37d97f5ceb54b1969bb828d5f4c1c68bef868905d60cc2
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=5ffa288e9a9acb38343427bedf8ebdc50c653813c20b8bb28625a135cc654efa
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully 

In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

So let’s start by creating topics, for example, I am using the popular 20 Newsgroups dataset provided by Scikit-Learn which has around 18,000 newsgroup articles on 20 topics. I will therefore select English as the main language for our documents:

In [18]:
model = BERTopic(calculate_probabilities=True)
topics, probs = model.fit_transform(docs)

Now let’s extract the topics with the most number of frequencies:

In [19]:
model.get_topic_freq().head()

,Topic,Count
1,-1,6887
0,0,1834
24,1,640
41,2,524
46,3,464


You can see -1 in the first row (index 0), -1 refers to all outliers and should generally be ignored. Next, let’s take a look at the most common topic generated:

In [20]:
topic_model.get_topic(0)[:10]

[('game', 0.010562552289888656),
 ('team', 0.009197001279897003),
 ('games', 0.007322072742198445),
 ('he', 0.007241745405015588),
 ('players', 0.006398398622812115),
 ('season', 0.006340396998154335),
 ('hockey', 0.0062259853942926565),
 ('play', 0.005885814506097448),
 ('25', 0.005759534007055261),
 ('year', 0.005722821565009483)]

The model is therefore stochastic, which means that the topics may differ from one run to another. Now let’s take a look at the topic probabilities to understand how safe BERTopic is that certain topics can be found in a document:

In [21]:
model.visualize_distribution(probs[0])

### Topic Reduction

Finally, we can also reduce the number of subjects after training a BERTopic model. The advantage of doing this is that you can decide the number of topics after knowing how many are actually created.

It is difficult to predict before training your model how many topics are in your documents and how many will be retrieved. Instead, we can decide afterwards how many subjects look realistic:

In [30]:
new_topics= model.reduce_topics(docs,nr_topics=30)

We can now use the update_topics function to update the subject representation with new parameters for the TF-IDF vectorization:

In [32]:
model.update_topics(docs, topics, n_gram_range=(1, 3))

### Topic Modelling

After training our model, we can use find_topics to search for topics similar to a search_term entry. Here we will be looking for topics that are closely related to the search term car. Next, we extract the most similar topic and check the results:

In [34]:
similar_topics, similarity = model.find_topics("car", top_n=5); similar_topics
model.get_topic(28)

[('ear', 0.04351919910159403),
 ('the ear', 0.024707498431836603),
 ('the', 0.01944052804787548),
 ('wax', 0.018332402237108674),
 ('to', 0.01810263325504848),
 ('ears', 0.01778350654801203),
 ('and', 0.014766975473806608),
 ('aids', 0.013569875426414652),
 ('with', 0.013430102829237324),
 ('hearing', 0.012698610903812234)]

Source:

https://thecleverprogrammer.com/2021/01/12/topic-modeling-with-machine-learning/